In [1]:
# Import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, accuracy_score, f1_score

from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, PassiveAggressiveClassifier, Perceptron, RidgeClassifier, RidgeClassifierCV, SGDClassifier

In [2]:
# Load the processed dataset
df = pd.read_csv('../data/preprocessed/with_diabetes_status/dataset_with_diabetes_status.csv')

In [3]:
# Drop the diabetes and hba1c columns
df = df.drop('diabetes', axis=1)

In [4]:
# Drop records where gender is 'Other'
df = df[df['gender'] != 'Other']

In [5]:
# One-hot encode categorical features
df = pd.get_dummies(df, columns=['gender', 'smoking_history'], drop_first=False)

In [6]:
# Define the order of categories for diabetes_status
status_order = ['non diabetic', 'stress induced prediabetic', 'stress induced type 2 diabetic', 'prediabetic', 'diabetic']

# Create a mapping for the specified order
status_mapping = {status: i for i, status in enumerate(status_order)}

# Map 'diabetes_status' to the numeric encoding
df['diabetes_status'] = df['diabetes_status'].map(status_mapping)

In [7]:
# Move the encoded 'diabetes_status' to the right
cols = df.columns.tolist()
cols.append(cols.pop(cols.index('diabetes_status')))
df = df[cols]

In [8]:
# Separate features and target
X = df.drop(columns=['diabetes_status'])
y = df['diabetes_status']

In [9]:
# Initialize SMOTE
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [10]:
# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [11]:
# List of models to evaluate
models = [
    LogisticRegression(),
    LogisticRegressionCV(),
    PassiveAggressiveClassifier(),
    Perceptron(),
    RidgeClassifier(),
    RidgeClassifierCV(),
    SGDClassifier(),
]

In [12]:
def evaluate_models(models, X, y, kf):
    results = {}
    
    for model in models:
        model_name = type(model).__name__
        print(f"Evaluating {model_name}...")
        
        fold_accuracies = []
        fold_f1_scores = []
        fold_reports = []
        
        for fold, (train_index, test_index) in enumerate(kf.split(X)):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            report = classification_report(y_test, y_pred)
            
            fold_accuracies.append(accuracy)
            fold_f1_scores.append(f1)
            fold_reports.append(report)
            
            print(f"Fold {fold + 1} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")
            print(f"Classification Report for Fold {fold + 1}:\n{report}\n")
        
        mean_accuracy = np.mean(fold_accuracies)
        std_accuracy = np.std(fold_accuracies)
        mean_f1 = np.mean(fold_f1_scores)
        std_f1 = np.std(fold_f1_scores)
        
        print(f"\n{model_name} - Mean Accuracy: {mean_accuracy:.4f} (± {std_accuracy:.4f}), Mean F1 Score: {mean_f1:.4f} (± {std_f1:.4f})\n")
        
        results[model_name] = {
            'mean_accuracy': mean_accuracy,
            'std_accuracy': std_accuracy,
            'mean_f1': mean_f1,
            'std_f1': std_f1,
            'reports': fold_reports
        }
    
    return results

In [13]:
# Call the function with the updated list of models, features, target, and KFold object
results = evaluate_models(models, X_res, y_res, kf)

Evaluating LogisticRegression...


C:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 1 - Accuracy: 0.4738, F1 Score: 0.4741
Classification Report for Fold 1:
              precision    recall  f1-score   support

           0       0.64      0.58      0.60      7583
           1       0.39      0.37      0.38      7703
           2       0.38      0.34      0.36      7362
           3       0.57      0.56      0.57      7626
           4       0.41      0.51      0.46      7755

    accuracy                           0.47     38029
   macro avg       0.48      0.47      0.47     38029
weighted avg       0.48      0.47      0.47     38029




C:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 2 - Accuracy: 0.4760, F1 Score: 0.4650
Classification Report for Fold 2:
              precision    recall  f1-score   support

           0       0.68      0.51      0.58      7650
           1       0.36      0.17      0.23      7541
           2       0.42      0.52      0.47      7678
           3       0.53      0.60      0.57      7631
           4       0.40      0.57      0.47      7529

    accuracy                           0.48     38029
   macro avg       0.48      0.48      0.46     38029
weighted avg       0.48      0.48      0.46     38029




...

Fold 1 - Accuracy: 0.7473, F1 Score: 0.7537
Classification Report for Fold 1:
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      7583
           1       0.58      0.71      0.64      7703
           2       0.50      0.63      0.56      7362
           3       0.86      0.53      0.66      7626
           4       0.99      0.88      0.93      7755

    accuracy                           0.75     38029
   macro avg       0.78      0.75      0.75     38029
weighted avg       0.78      0.75      0.75     38029


Fold 2 - Accuracy: 0.7617, F1 Score: 0.7660
Classification Report for Fold 2:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      7650
           1       0.58      0.72      0.64      7541
           2       0.54      0.62      0.58      7678
           3       0.87      0.58      0.70      7631
           4       0.96      0.90      0.93      7529

    accuracy                

C:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Fold 4 - Accuracy: 0.7159, F1 Score: 0.7108
Classification Report for Fold 4:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7479
           1       0.59      0.41      0.48      7676
           2       0.46      0.85      0.59      7662
           3       0.94      0.41      0.57      7556
           4       0.95      0.92      0.94      7656

    accuracy                           0.72     38029
   macro avg       0.78      0.72      0.71     38029
weighted avg       0.78      0.72      0.71     38029




C:\Users\david\anaconda3\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:744: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


Fold 5 - Accuracy: 0.7690, F1 Score: 0.7715
Classification Report for Fold 5:
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      7634
           1       0.58      0.78      0.66      7524
           2       0.57      0.59      0.58      7613
           3       0.89      0.58      0.70      7672
           4       0.96      0.91      0.93      7586

    accuracy                           0.77     38029
   macro avg       0.79      0.77      0.77     38029
weighted avg       0.79      0.77      0.77     38029



SGDClassifier - Mean Accuracy: 0.7353 (± 0.0321), Mean F1 Score: 0.7278 (± 0.0501)

